In [ ]:
import pandas as pd
import joblib
from data_preprocessing import load_data, preprocess_data
from feature_engineering import create_features, scale_features, split_data
from model_training import train_model, save_model
from inference import predict, evaluate_model

ModuleNotFoundError: No module named 'src'

In [ ]:
def run_pipeline(data_path, model_name="LinearRegression", test_size=0.3, random_state=42):
    print("--- Запуск пайплайна прогнозирования доходов бюджета ---")

    # 1. Загрузка и предобработка данных
    print("1. Загрузка и предобработка данных...")
    df = load_data(data_path)
    processed_df = preprocess_data(df.copy())
    print("   Данные успешно предобработаны.")

    # 2. Инженерия признаков и разделение данных
    print("2. Инженерия признаков и разделение данных...")
    X, y = create_features(processed_df.copy())
    X_train, X_test, y_train, y_test = split_data(X, y, test_size=test_size, random_state=random_state)
    X_train_scaled, X_test_scaled, scaler = scale_features(X_train, X_test)
    print("   Признаки созданы и данные разделены.")

    # 3. Обучение модели
    print(f"3. Обучение модели: {model_name}...")
    model = train_model(model_name, X_train_scaled, y_train)
    print(f"   Модель {model_name} успешно обучена.")

    # 4. Сохранение модели и скейлера
    print("4. Сохранение модели и скейлера...")
    model_save_path = f"models/{model_name.lower()}_model.joblib"
    scaler_save_path = "models/scaler.joblib"
    import os
    os.makedirs("models", exist_ok=True)
    save_model(model, model_save_path)
    joblib.dump(scaler, scaler_save_path)
    print(f"   Модель сохранена в {model_save_path}")
    print(f"   Скейлер сохранен в {scaler_save_path}")

    # 5. Оценка модели
    print("5. Оценка модели...")
    y_pred = predict(model, scaler, X_test)
    metrics = evaluate_model(y_test, y_pred)
    print("   Метрики оценки модели:")
    for metric, value in metrics.items():
        print(f"   - {metric}: {value:.4f}")
    print("--- Пайплайн завершен ---")

    return model, scaler, metrics

if __name__ == "__main__":
    # Пример использования:
    # Убедитесь, что файл dataset.csv находится в папке data/
    # Или укажите полный путь к вашему файлу данных
    data_file = "data/dataset.csv" # Предполагается, что файл dataset.csv находится в корневой папке или в папке data

    # Проверим существование файла данных
    import os
    if not os.path.exists(data_file):
        print(f"Ошибка: Файл данных не найден по пути {data_file}. Пожалуйста, убедитесь, что он существует.")
    else:
        # Запуск пайплайна с Linear Regression
        print("\nЗапуск пайплайна с Linear Regression:")
        lr_model, lr_scaler, lr_metrics = run_pipeline(data_file, model_name="LinearRegression")

        # Запуск пайплайна с RandomForestRegressor
        print("\nЗапуск пайплайна с RandomForestRegressor:")
        rf_model, rf_scaler, rf_metrics = run_pipeline(data_file, model_name="RandomForestRegressor")

        # Запуск пайплайна с XGBRegressor
        print("\nЗапуск пайплайна с XGBRegressor:")
        xgb_model, xgb_scaler, xgb_metrics = run_pipeline(data_file, model_name="XGBRegressor")